# Аналитика по продажам игр разных жанров и платформ

- Автор: Диана Лукинова
- Дата: 13.11.2025

### Цели и задачи проекта

Цель проекта: сделать обзор игровых платформ, изучить объёмы продаж игр разных жанров и региональные предпочтения игроков.

Задачи проекта:
познакомиться с данными, проверить их корректность и провести предобработку, получив необходимый срез данных
- Выгрузить данные, ознакомиться с ними
- Предобработать данные: проверить на пропуски, неверные типы данных, дубликаты
- Отфильтровать данные с 2000 по 2013 год
- Разделить данные на категории в зависимости от оценок пользователей или критиков
- Выделить топ-7 платформ по количеству игр, выпущенных за актуальный период

### Описание данных

<font color='#777778'>
Данные содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:

Name — название игры.

Platform — название платформы.

Year of Release — год выпуска игры.

Genre — жанр игры.

NA sales — продажи в Северной Америке (в миллионах проданных копий).

EU sales — продажи в Европе (в миллионах проданных копий).

JP sales — продажи в Японии (в миллионах проданных копий).

Other sales — продажи в других странах (в миллионах проданных копий).

Critic Score — оценка критиков (от 0 до 100).

User Score — оценка пользователей (от 0 до 10).

Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board).</font>

### Содержимое проекта

- Загрузка данных и знакомство с ними
- Проверка ошибок в данных и их предобработка
    - Названия, или метки, столбцов датафрейма
    - Типы данных
    - Наличие пропусков в данных
    - Явные и неявные дубликаты
- Фильтрация данных
- Категоризация данных
- Общий вывод
---

## 1. Загрузка данных и знакомство с ними

- Загрузите необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
#df = pd.read_csv('/datasets/new_games.csv')

In [2]:
import pandas as pd
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

- Познакомьтесь с данными: выведите первые строки и результат метода `info()`.


In [3]:
df.head(10)

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.5,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.7,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [5]:
shape_before_work = df.shape
shape_before_work

(16956, 11)

- В предоставленных данных 11 столбцов и 16956 строк. Данные соответствуют заявленному описанию. Предварительно были обнаружены пропуски в столбцах Name, Year of Release, Genre, Critic Score, User Score, Rating. 
- Не во всех столбцах представлены правильные типы данных: неверные типы встречаются в столбцах Year of Release (нужен int64),  EU sales, JP sales, User Score (нужен float64).
- Названия столбцов отражают содержащиеся в них данные. Однако было бы удобнее для работы привести названия столбцов к формату snake case

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

In [6]:
#Названия всех столбцов датафрейма
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

Стиль написания не соответствует стилю snake case

In [7]:
#Приводим названия к стилю snake case
df.columns = ['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating']
df.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.5,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.7,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [8]:
# создаем копию датасета до преобразования для возможности проверить сделанные изменения после предобработки
tmp = df.copy() 
len(tmp)

16956

### 2.2. Типы данных

Судя по данным метода info() в данных встречаются некорректные типы.

Выведем уникальные значения в этих столбцах

In [9]:
for i in ['jp_sales', 'eu_sales', 'user_score', 'year_of_release']:
    unique_v = df[i].unique()
    print(f'{i} : {unique_v}')

jp_sales : ['3.77' '6.81' '3.79' '3.28' '10.22' '4.22' '6.5' '2.93' '4.7' '0.28'
 '1.93' '4.13' '7.2' '3.6' '0.24' '2.53' '0.98' '0.41' '3.54' '4.16'
 '6.04' '4.18' '3.84' '0.06' '0.47' '5.38' '5.32' '5.65' '1.87' '0.13'
 '3.12' '0.36' '0.11' '4.35' '0.65' '0.07' '0.08' '0.49' '0.3' '2.66'
 '2.69' '0.48' '0.38' '5.33' '1.91' '3.96' '3.1' '1.1' '1.2' '0.14' '2.54'
 '2.14' '0.81' '2.12' '0.44' '3.15' '1.25' '0.04' '0.0' '2.47' '2.23'
 '1.69' '0.01' '3.0' '0.02' '4.39' '1.98' '0.1' '3.81' '0.05' '2.49'
 '1.58' '3.14' '2.73' '0.66' '0.22' '3.63' '1.45' '1.31' '2.43' '0.7'
 '0.35' '1.4' '0.6' '2.26' '1.42' '1.28' '1.39' '0.87' '0.17' '0.94'
 '0.19' '0.21' '1.6' '0.16' '1.03' '0.25' '2.06' '1.49' '1.29' '0.09'
 '2.87' '0.03' '0.78' '0.83' '2.33' '2.02' '1.36' '1.81' '1.97' '0.91'
 '0.99' '0.95' '2.0' '1.01' '2.78' '2.11' '1.09' '0.2' '1.9' '1.27' '3.61'
 '1.57' '2.2' '1.7' '1.08' '0.15' '1.11' '0.29' '1.54' '0.12' '0.89'
 '4.87' '1.52' '1.32' '1.15' '4.1' '1.46' '0.46' '1.05' '1.61' '0.26'
 

Судя по уникальным значениям столбцов с неверными типами данных, то в столбцах jp_sales и eu_sales встречаются строковые значения 'unknown', а в столбце user_score встречается строковое значение 'tbd'. В столбце year_of_release тип float64, а не int64, потому что в этом столбце есть пропуски. 

Обработаем строковые значения и исправим неверные типы данных

In [10]:
#заменим строковые значения в числовых столбцах на пропуски
df[['jp_sales', 'eu_sales']] = df[['jp_sales', 'eu_sales']].replace('unknown', np.nan)
df['user_score'] = df['user_score'].replace('tbd', np.nan)

In [11]:
#некорректный тип в столбцах eu_sales, jp_sales нужно поменять на float64. Строковое значение 'unknown' было заменено на None в предыдущем пункте 
df[['jp_sales', 'eu_sales']] = df[['jp_sales', 'eu_sales']].astype('float64')

In [12]:
#некорректный тип в столбце user_score нужно поменять на float64. Строковое значение 'tbd' было заменено на None в предыдущем пункте 
df['user_score'] = df['user_score'].astype('float64')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


Тип данных в столбце year_of_release будет заменён на int64 после обработки пропусков

### 2.3. Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [14]:
#Количество пропусков в каждом столбце в абсолютных значениях
df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [15]:
#Количество пропусков в каждом столбце в относительных значениях
df.isna().sum() / df.shape[0]

name               0.000118
platform           0.000000
year_of_release    0.016218
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64

In [ ]:
def show_missing_stats(tmp0):
    """
    Функция для отображения статистики пропущенных значений в DataFrame.
    """
    missing_stats = pd.DataFrame({
        'Кол-во пропусков': tmp0.isnull().sum(),
        'Доля пропусков': tmp0.isnull().mean()
    })
    missing_stats = missing_stats[missing_stats['Кол-во пропусков'] > 0]
    
    if missing_stats.empty:
        return "Пропусков в данных нет"
    
    # Форматируем при выводе через Styler
    return (missing_stats.style.format({'Доля пропусков': '{:.4f}'}).background_gradient(cmap='coolwarm'))
show_missing_stats(df)

,Кол-во пропусков,Доля пропусков
name,2,0.0001
year_of_release,275,0.0162
genre,2,0.0001
eu_sales,6,0.0004
jp_sales,4,0.0002
critic_score,8714,0.5139
user_score,9268,0.5466
rating,6871,0.4052


### ВЫВОД

Пропуски характерны для столбцов critic_score, user_score и rating (почти половина значений пропущена). Эти значения могут отсутствовать просто потому, что игра ещё не успела получить свои оценки от пользователей или критиков. Оценка рейтинга организации ESRB может отсутствовать, если игра выпущена не в США или Канаде. 

Строки с пропусками в столбцах name, year_of_release будут удалены (в первых двух всего по 2 пропуска, поэтому удаление не будет критичным для результатов). Для столбца с годом такое решение будет оптимальным, так как при отборе данных по году выпуска игры, мы всё равно возьмём лишь те игры, у которых известен год выпуска, находящийся в диапазоне от 2000 до 2013.

Для столбцов critic_score, user_score мы оставим пропуски. А для столбца rating на месте пропусков мы поставим значение 'unknown', чтобы отделить игры, выпущенные для США и Канады от всех остальных.

In [17]:
df = df.dropna(subset=['name', 'genre', 'year_of_release'])

In [18]:
#после удаления строк с пропусками в столбце с годом выпуска приведём данные к типу int64
df['year_of_release'] = df['year_of_release'].astype('int64')

In [19]:
#Заменяем получившиеся пропуски в столбцах eu_sales, jp_sales
def mean_(row):
    if pd.isna(row['eu_sales']):
        group = df[(df['platform'] == row['platform']) & 
                (df['year_of_release'] == row['year_of_release'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']
df['eu_sales'] = df.apply(mean_, axis=1)

def mean_(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['platform'] == row['platform']) & 
                (df['year_of_release'] == row['year_of_release'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']
df['jp_sales'] = df.apply(mean_, axis=1)

In [20]:
# заменяем пропуски в столбце rating на значение unknown
df['rating'] = df['rating'].fillna('unknown')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  int64  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  float64
 6   jp_sales         16679 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     8085 non-null   float64
 9   user_score       7558 non-null   float64
 10  rating           16679 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


теперь все типы данных корректны, а пропуски обработаны

In [22]:

# Посмотрим какие пропуски остались
show_missing_stats(df)

,Кол-во пропусков,Доля пропусков
critic_score,8594,0.5153
user_score,9121,0.5469


### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [23]:
for i in ['genre', 'platform', 'rating', 'year_of_release']:
    unique_values = df[i].unique()
    print(f'{i} : {unique_values}')

genre : ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']
platform : ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
rating : ['E' 'unknown' 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
year_of_release : [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000 1995 1991 1981 1987 1980 1983]


В столбце genre встречаются одни и те же значения в разном регистре, приведём их к одному регистру.

In [24]:
df['genre'] = df['genre'].str.lower()
df['name'] = df['name'].str.lower()
df['rating'] = df['rating'].str.upper()

In [25]:
for i in ['genre', 'platform', 'rating', 'year_of_release']:
    unique_values = df[i].unique()
    print(f'{i} : {unique_values}')

genre : ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']
platform : ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
rating : ['E' 'UNKNOWN' 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
year_of_release : [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000 1995 1991 1981 1987 1980 1983]


- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

In [26]:
df.nunique()

name               11426
platform              31
year_of_release       37
genre                 12
na_sales             401
eu_sales             313
jp_sales             248
other_sales          155
critic_score          81
user_score            95
rating                 9
dtype: int64

In [27]:
df.duplicated(subset=None, keep='first').sum()

235

Явных дубликатов 235. Эти строки следует удалить

In [28]:
df_sorted = df.sort_values(by=df.columns.tolist())
df_cleaned = df_sorted.drop_duplicates()
df_cleaned.duplicated().sum()

0

- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

In [29]:
print(f'Было строк в исходных данных: {shape_before_work[0]}')
print(f'Стало строк в предобработанном датасете: {df_cleaned.shape[0]}')
print(f'Удалено строк: {shape_before_work[0] - df_cleaned.shape[0]}') #абсолютное значение количества удалённых строк
#number_of_deleted_str_rel = (shape_before_work[0] - df_cleaned.shape[0]) / df_cleaned.shape[0]
number_of_deleted_str_rel = (shape_before_work[0] - df_cleaned.shape[0]) / shape_before_work[0]
print(f'Процент удалённых строк: {round(number_of_deleted_str_rel * 100, 2)}') #относительное значение количества удалённых строк

Было строк в исходных данных: 16956
Стало строк в предобработанном датасете: 16444
Удалено строк: 512
Процент удалённых строк: 3.02


In [30]:
# Комментарий ревьюера 2
# Проверим сколько удалено строк датасета
a, b = len(tmp), len(df_cleaned)
print(" Было строк в исходном датасете", a,
      '\n', "Осталось строк в датасете после обработки", b,
      '\n', "Удалено строк в датасете после обработки", a-b,
      '\n', "Процент потерь", round((a-b)/a*100, 2))

 Было строк в исходном датасете 16956 
 Осталось строк в датасете после обработки 16444 
 Удалено строк в датасете после обработки 512 
 Процент потерь 3.02


Всего было удалено чуть более 3 процентов всех строк исходных данных

В ходе предобработки данных были найдены пропуски, явные и неявные дубликаты, а также неверные типы данных в нескольких столбцах. Всего было удалено 512 строк с дубликатами и пропусками, типы данных были заменены в четырёх столбцах. Название столбцов были приведены к стилю snake case, а значения в столбцах name, genre были приведены к нижнему регистру для устранения неявных дубликатов 

---

## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [31]:
df_actual = df_cleaned.loc[(df_cleaned['year_of_release'] >= 2000) & (df_cleaned['year_of_release'] <= 2013)]

In [32]:
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
3394,frozen: olaf's quest,3DS,2013,platform,0.27,0.27,0.00,0.05,NaN,NaN,UNKNOWN
3906,frozen: olaf's quest,DS,2013,platform,0.21,0.26,0.00,0.04,NaN,NaN,UNKNOWN
2478,tales of xillia 2,PS3,2012,role-playing,0.20,0.12,0.45,0.07,71.0,7.9,T
8460,.hack//g.u. vol.1//rebirth,PS2,2006,role-playing,0.00,0.00,0.17,0.00,NaN,NaN,UNKNOWN
7182,.hack//g.u. vol.2//reminisce,PS2,2006,role-playing,0.11,0.09,0.00,0.03,NaN,NaN,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...
7233,zumba fitness: world party,Wii,2013,misc,0.11,0.10,0.00,0.02,NaN,NaN,E
6970,zumba fitness: world party,XOne,2013,misc,0.17,0.05,0.00,0.02,73.0,6.2,E
15739,zwei!!,PSP,2008,role-playing,0.00,0.00,0.02,0.00,NaN,NaN,UNKNOWN
13138,zyuden sentai kyoryuger: game de gaburincho!!,3DS,2013,action,0.00,0.00,0.05,0.00,NaN,NaN,UNKNOWN


In [ ]:

df_actual.info(), df_actual.year_of_release.sort_values().unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 3394 to 9260
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  int64  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12781 non-null  float64
 6   jp_sales         12781 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     7169 non-null   float64
 9   user_score       6483 non-null   float64
 10  rating           12781 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.2+ MB


(None,
 array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
        2011, 2012, 2013]))

In [34]:
df_actual.info(), df_actual.year_of_release.sort_values().unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 3394 to 9260
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  int64  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12781 non-null  float64
 6   jp_sales         12781 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     7169 non-null   float64
 9   user_score       6483 non-null   float64
 10  rating           12781 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.2+ MB


(None,
 array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
        2011, 2012, 2013]))

---

## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [35]:
df_actual['user_score_category'] = pd.cut(df_actual['user_score'], bins=[0, 3, 8, 10], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

In [36]:
df_actual['critic_score_category'] = pd.cut(df_actual['critic_score'], bins=[0, 30, 80, 100], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

In [37]:
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
3394,frozen: olaf's quest,3DS,2013,platform,0.27,0.27,0.00,0.05,NaN,NaN,UNKNOWN,NaN,NaN
3906,frozen: olaf's quest,DS,2013,platform,0.21,0.26,0.00,0.04,NaN,NaN,UNKNOWN,NaN,NaN
2478,tales of xillia 2,PS3,2012,role-playing,0.20,0.12,0.45,0.07,71.0,7.9,T,средняя оценка,средняя оценка
8460,.hack//g.u. vol.1//rebirth,PS2,2006,role-playing,0.00,0.00,0.17,0.00,NaN,NaN,UNKNOWN,NaN,NaN
7182,.hack//g.u. vol.2//reminisce,PS2,2006,role-playing,0.11,0.09,0.00,0.03,NaN,NaN,UNKNOWN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7233,zumba fitness: world party,Wii,2013,misc,0.11,0.10,0.00,0.02,NaN,NaN,E,NaN,NaN
6970,zumba fitness: world party,XOne,2013,misc,0.17,0.05,0.00,0.02,73.0,6.2,E,средняя оценка,средняя оценка
15739,zwei!!,PSP,2008,role-playing,0.00,0.00,0.02,0.00,NaN,NaN,UNKNOWN,NaN,NaN
13138,zyuden sentai kyoryuger: game de gaburincho!!,3DS,2013,action,0.00,0.00,0.05,0.00,NaN,NaN,UNKNOWN,NaN,NaN


Группировка данных по выделенным категориям, количество игр в каждой категории оценок

In [38]:
df_actual.groupby('user_score_category')['name'].count()

user_score_category
низкая оценка      116
средняя оценка    4081
высокая оценка    2286
Name: name, dtype: int64

In [39]:
df_actual.groupby('critic_score_category')['name'].count()

critic_score_category
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: name, dtype: int64

Топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [40]:
df_actual.groupby('platform')['name'].count().sort_values(ascending=False).head(7)

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: name, dtype: int64

In [41]:
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 3394 to 9260
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   name                   12781 non-null  object  
 1   platform               12781 non-null  object  
 2   year_of_release        12781 non-null  int64   
 3   genre                  12781 non-null  object  
 4   na_sales               12781 non-null  float64 
 5   eu_sales               12781 non-null  float64 
 6   jp_sales               12781 non-null  float64 
 7   other_sales            12781 non-null  float64 
 8   critic_score           7169 non-null   float64 
 9   user_score             6483 non-null   float64 
 10  rating                 12781 non-null  object  
 11  user_score_category    6483 non-null   category
 12  critic_score_category  7169 non-null   category
dtypes: category(2), float64(6), int64(1), object(4)
memory usage: 1.2+ MB


---

## 5. Итоговый вывод



В проекте были предобработаны данные датасета: названия столбцов заменены на соответствующие стилю snake case, удалены или заменены строки с пропусками (кроме пропусков в столбцах с оценками пользователей и критиков), удалены строки с явными и неявными дубликатами. Также были изменены типы данных четырёх столбцов на корректные (Year of Release - int64,  EU sales, JP sales, User Score - float64).

Предобработанный датасет включает в себя 12781 строку и 13 столбцов.

Выводы:
### Типы данных
Некорректные типы данных вызваны тем, что в столбцах с числовыми значениями встречаются строковые значения (EU sales, JP sales, User Score); или тем, что в столбцах с целыми числами встречаются пропуски (Year of Release). Некорректные значения были заменены на пропуски, строки с пропусками в столбце Year of Release были удалены. В результате предобработки все столбцы получили корректные типы данных.

### Наличие пропусков в данных
Пропуски характерны для столбцов critic_score, user_score и rating (почти половина значений пропущена). Эти значения могут отсутствовать просто потому, что игра ещё не успела получить свои оценки от пользователей или критиков. Оценка рейтинга организации ESRB может отсутствовать, если игра выпущена не в США или Канаде. 

### Явные и неявные дубликаты в данных
В столбце genre встретились одни и те же значения в разном регистре, мы привели их к нижнему регистру. После преобразования неявных дубликатов явных дубликатов оказалось 235, эти строки были удалены.
Всего было удалено 512 строк с пропусками или дубликатами. 

### Фильтрация данных
После фильтрации данных датасет включал в себя 12781 строк, где были отобраны игры, выпущенные с 2000 по 2013 годы включительно.

### Категоризация данных
В ходе проекта была проведена фильтрация данных: были отобраны игры, выпущенные с 2000 по 2013 годы включительно. Также было добавлено два новых столбца с категориями оценок пользователей и критиков 'user_score_category', 'critic_score_category', в которых оценки пользователей были разделены на категории "низкая оценка", "средняя оценка", "высокая оценка". 

По оценкам пользователей низкие оценки получили 116 игр, средние - 4081 игр, высокие - 2286 игр. 
По оценкам критиков низкие оценки получили 55 игр, средние - 5422 игр, высокие - 1692 игры. Таким образом, пользователи больше склонны ставить высокие оценки, чем критики.

Также в проекте были выделены топ-7 платформ по количеству игр, выпущенных за указанный период (PS2, DS, Wii, PSP, X360, PS3, GBA). Наиболее значимые по количеству игр платформы PS2 (2127 игр) и DS (2120 игр).